# pytorch与tensorflow的区别 
+ torch: 简单和灵活
+ 张量：可以在GPU或其他专用硬件上运行，加速效果

# 利用torch解决线性回归问题 

## 完整代码

In [14]:
import numpy as np
import torch
import torch.nn as nn

#torch里要求的数据类型必须是float
#函数返回一个有终点和起点的固定步长的排列
x = np.arange(1,12,dtype = np.float32).reshape(-1,1)
y = 2 * x + 3

##重新定义线性回归 继承nn.module，实现前向传播
class LinearRegressionModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super().__init__()#继承父类的方法
        self.linear = nn.Linear(input_dim,output_dim) #定义全连接层
        
    def forward(self,inp):
        out = self.linear(inp) 
        return out
    
regression_model = LinearRegressionModel(1,1)

#指定模型的训练方式 CPU or GPU  注意是cuda不是cude
device = torch.device("cuda:0"if torch.cuda.is_available() else "cpu")
regression_model.to(device)

#设置参数
epochs = 1000  # 训练次数
learning_rate = 0.01  # 学习速率
optimizer = torch.optim.SGD(regression_model.parameters(), learning_rate)  # 优化器（未来会详细介绍），这里使用随机梯度下降算法（SGD）
criterion = nn.MSELoss()  # 使用均方误差定义损失函数

#进行训练
for epoch in range(epochs):
    #数据类型的转换
    inputs = torch.from_numpy(x).to(device)
    labels = torch.from_numpy(y).to(device)
    
    #训练
    optimizer.zero_grad() #每次求偏导都会清零，否则会进行叠加
    outputs = regression_model(inputs)
    loss = criterion(outputs,labels) #通过均方误差评估预测误差
    loss.backward() #反向传播
    optimizer.step() #更新权重参数
    
    if epoch % 50 == 0:
        print("epoch:",epoch,"loss:",loss.item())

epoch: 0 loss: 148.56253051757812
epoch: 50 loss: 1.5162343978881836
epoch: 100 loss: 0.9879584312438965
epoch: 150 loss: 0.6436681747436523
epoch: 200 loss: 0.4193764626979828
epoch: 250 loss: 0.2732224464416504
epoch: 300 loss: 0.1780959814786911
epoch: 350 loss: 0.1160411462187767
epoch: 400 loss: 0.07564301788806915
epoch: 450 loss: 0.04923228919506073
epoch: 500 loss: 0.03211582452058792
epoch: 550 loss: 0.020908044651150703
epoch: 600 loss: 0.013639013282954693
epoch: 650 loss: 0.00891256332397461
epoch: 700 loss: 0.005786513909697533
epoch: 750 loss: 0.003886700375005603
epoch: 800 loss: 0.0024582520127296448
epoch: 850 loss: 0.0016097445040941238
epoch: 900 loss: 0.0010669843759387732
epoch: 950 loss: 0.00077777449041605


## 函数详细讲解 

### reshape 
+ numpy.arange(n).reshape(a, b) 依次生成n个自然数，并且以a行b列的数组形式显示
+ mat (or array).reshape(c, -1) 必须是矩阵格式或者数组格式，才能使用 .reshape(c, -1) 函数
    + 1的作用就在此: **自动计算d：d=数组或者矩阵里面所有的元素个数/c**, d必须是整数，不然报错
+ 常见用法：
    + reshape(1,-1)转化成1行
    + reshape(2,-1)转换成两行
    + reshape(-1,1)转换成1列：

In [4]:
x = np.arange(1,12,dtype = np.float32)
x.reshape(-1,1)

array([[ 1.],
       [ 2.],
       [ 3.],
       [ 4.],
       [ 5.],
       [ 6.],
       [ 7.],
       [ 8.],
       [ 9.],
       [10.],
       [11.]], dtype=float32)

### torch.device() 

In [ ]:
#这行代码的意思是将所有最开始读取数据时的tensor变量copy一份到device所指定的GPU上去，
# 之后的运算都在GPU上进行。
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

##多个GPU的使用方法
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Model()
if torch.cuda.device_count() > 1:#device_count计算GPU个数
    model = nn.DataParallel(model，device_ids=[0,1,2])
model.to(device）

### Tensor总结
+ Tensor 和 Numpy都是矩阵，区别是前者可以在GPU上运行，后者只能在CPU上；
+ Tensor和Numpy互相转化很方便，类型也比较兼容
+ Tensor可以直接通过print显示数据类型，而Numpy不可以